In [ ]:
# change into the root directory of the project
import os
if os.getcwd().split("/")[-1] == "dev":
    os.chdir('..')
    
# get the current directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))


# Set path, where to save files of tables
PATH = cwd
HDF_PATH = PATH + '/data/hdf/'
IMG_PATH = PATH + '/images/05/'
print(HDF_PATH)


import numpy as np
import scipy
from scipy import stats


from neurolib.models.wc-adap import WCModel
from neurolib.models.wc_input import WCModel_input

from neurolib.utils.parameterSpace import ParameterSpace
from neurolib.optimize.exploration import BoxSearch
import neurolib.utils.functions as func
import neurolib.utils.devutils as du

#import neurolib.utils.brainplot as bp
import neurolib.optimize.exploration.explorationUtils as eu

from neurolib.utils.loadData import Dataset

from Derivations import Derivations

#for the brainplot functions
from Brainplot import plot_brain
from Brainplot import plot_involvement_distribution
import xarray as xr
from neurolib.utils.signal import Signal
import matplotlib.pyplot as plt

In [ ]:
#Packages for plotting
from plotly.offline import init_notebook_mode, plot, iplot
from plotly.subplots import make_subplots
import plotly.offline as pyo

import chart_studio.plotly as py #chart_studio has to be installed: pip install chart_studio
import plotly.graph_objs as go
import plotly.figure_factory as ff
import plotly.express as px

In [ ]:
from Templates import template
from Templates import brain_result_color_list
from Templates import brain_result_colors

In [ ]:
colors = brain_result_colors

width = template.layout.width * (1/3)
height = template.layout.height * (2/3)

In [ ]:
ds = Dataset("gw", fcd=True)

In [ ]:
wc = WCModel(Cmat = ds.Cmat, Dmat = ds.Dmat)
wc_input=WCModel_input(Cmat = ds.Cmat, Dmat = ds.Dmat)

# 1.For critical value

In [ ]:
fix = {'duration': 360.0*1000, 'sigma_ou': 0.287031, 'K_gl': 2.573845, 
      # 'adap_init': 0.0 * np.random.uniform(0.99, 1, (1, 1)), #inkl a_adap Null setzen für ausschalten der Adap
       'tau_exc': 2.5, 'tau_inh': 3.75, 
       'c_excexc': 16, 'c_excinh': 12, 'c_inhexc': 12, 'c_inhinh': 3, 
       'a_exc': 1.0, 'a_inh': 1.0, 
       'mu_exc': 5.0, 'mu_inh': 5.0,
       'a_adap': 98.381822, 'tau_adap': 3050.402224,
       'exc_ext': 4.960871, 'inh_ext': 2.668888}
wc.params.update(fix)

In [ ]:
dev = Derivations(model=wc, model_input=wc_input, params=fix)

In [ ]:
wc.run()

In [ ]:
thresh = 0.2 * np.max(wc.output, axis=1)
x=wc.exc[:,-600000:]
states_critical = [dev.getUpDownWindows(x[k], thresh[k], filter_long=True, dur=25) for k in range(len(x))]

In [ ]:
fig = go.Figure()

time = len(states_critical[0]) / (1000/0.1) # number_of_integration_steps / (ms/integration_time_constant)
steps = len(states_critical[0])
fig.add_trace(go.Heatmap(z=states_critical, x=np.linspace(0,time,steps),
                         colorscale=[[0.0, colors['DOWN']], [0.5, colors['DOWN']], 
                                     [0.5, colors['UP']], [1.0, colors['UP']]],
                         colorbar=dict(nticks=2,
                                       tickvals=[0.05, 0.95], 
                                       ticktext=['DOWN', 'UP'],
                                       tickangle=90)))

fig.update_layout(template=template,
                  annotations=[
                      dict(x=0.49, y=1.15, text='b=98.38', font_color='gray')
                  ],
                 width=width*2, height=height)
fig.update_xaxes(tickvals=[], ticktext=[], ticks='')
fig.update_yaxes(title_text='Node', tickvals=[0,19,39,59,79], ticktext=['1','20','40','60','80'])
fig.show()

In [ ]:
fig.write_img(IMG_PATH + 'b_critical.png')

# 2. For half critical value

In [ ]:
fix = {'duration': 360.0*1000, 'sigma_ou': 0.287031, 'K_gl': 2.573845, 
      # 'adap_init': 0.0 * np.random.uniform(0.99, 1, (1, 1)), #inkl a_adap Null setzen für ausschalten der Adap
       'tau_exc': 2.5, 'tau_inh': 3.75, 
       'c_excexc': 16, 'c_excinh': 12, 'c_inhexc': 12, 'c_inhinh': 3, 
       'a_exc': 1.0, 'a_inh': 1.0, 
       'mu_exc': 5.0, 'mu_inh': 5.0,
       'a_adap': 60, 'tau_adap': 3050.402224,
       'exc_ext': 4.960871, 'inh_ext': 2.668888}
wc.params.update(fix)

In [ ]:
dev = Derivations(model=wc, model_input=wc_input, params=fix)

In [ ]:
wc.run()

In [ ]:
thresh = 0.2 * np.max(wc.output, axis=1)
x=wc.exc[:,-600000:]
states_half = [dev.getUpDownWindows(x[k], thresh[k], filter_long=True, dur=25) for k in range(len(x))]

In [ ]:
fig = go.Figure()

time = len(states_half[0]) / (1000/0.1) # number_of_integration_steps / (ms/integration_time_constant)
steps = len(states_half[0])
fig.add_trace(go.Heatmap(z=states_half, x=np.linspace(0,time,steps),
                         colorscale=[[0.0, colors['DOWN']], [0.5, colors['DOWN']], 
                                     [0.5, colors['UP']], [1.0, colors['UP']]],#[(0.00, colors['DOWN']),(1.0, colors['UP'])],
                         colorbar=dict(nticks=2,
                                       tickvals=[0.05, 0.95], 
                                       ticktext=['DOWN', 'UP'],
                                       tickangle=90)))

fig.update_layout(template=template,
                  annotations=[
                      dict(x=0.49, y=1.15, text='b=60', font_color='gray')
                  ],
                  width=800, height=400)
fig.update_xaxes(tickvals=[], ticktext=[], ticks='')
fig.update_yaxes(title_text='Node', tickvals=[0,19,39,59,79], ticktext=['1','20','40','60','80'])
fig.show()

In [ ]:
fig.write_img(IMG_PATH + 'b_half.png')

# 3. For b higher

In [ ]:
fix = {'duration': 360.0*1000, 'sigma_ou': 0.287031, 'K_gl': 2.573845, 
      # 'adap_init': 0.0 * np.random.uniform(0.99, 1, (1, 1)), #inkl a_adap Null setzen für ausschalten der Adap
       'tau_exc': 2.5, 'tau_inh': 3.75, 
       'c_excexc': 16, 'c_excinh': 12, 'c_inhexc': 12, 'c_inhinh': 3, 
       'a_exc': 1.0, 'a_inh': 1.0, 
       'mu_exc': 5.0, 'mu_inh': 5.0,
       'a_adap': 120, 'tau_adap': 3050.402224,
       'exc_ext': 4.960871, 'inh_ext': 2.668888}
wc.params.update(fix)

In [ ]:
dev = Derivations(model=wc, model_input=wc_input, params=fix)

In [ ]:
wc.run()

In [ ]:
thresh = 0.2 * np.max(wc.output, axis=1)
x=wc.exc[:,-600000:]
states_higher = [dev.getUpDownWindows(x[k], thresh[k], filter_long=True, dur=25) for k in range(len(x))]

In [ ]:
fig = go.Figure()

time = len(states_higher[0]) / (1000/0.1) # number_of_integration_steps / (ms/integration_time_constant)
steps = len(states_higher[0])
fig.add_trace(go.Heatmap(z=states_higher, x=np.linspace(0,time,steps),
                         colorscale=[[0.0, colors['DOWN']], [0.5, colors['DOWN']], 
                                     [0.5, colors['UP']], [1.0, colors['UP']]],
                         colorbar=dict(nticks=2,
                                       tickvals=[0.05, 0.95], 
                                       ticktext=['DOWN', 'UP'],
                                       tickangle=90)))

fig.update_layout(template=template,
                  annotations=[
                      dict(x=0.49, y=1.15, text='b=120', font_color='gray')
                  ],
                 width=width*2, height=height)
fig.update_xaxes(title_text='Time [s]', tickvals=[0,20,40,60])
fig.update_yaxes(title_text='Node', tickvals=[0,19,39,59,79], ticktext=['1','20','40','60','80'])
fig.show()